In [333]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import math
import sklearn
from scipy import stats
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from sklearn import linear_model
import patsy
import statsmodels.formula.api as smf
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from __future__ import division
%config InlineBackend.figure_format = 
'png' #set 'png' here when working on notebook
%matplotlib inline

# Classification Model

# Q1 Baseline Model 

In [304]:
data = pd.read_excel("NRDF.xlsx")
SaleOver2k = (np.array([data['SalePrice']>200000])*1).reshape(len(data),1)
data = data.assign(SaleOver2k = SaleOver2k)

In [330]:
train, test = train_test_split(data, test_size=0.2, random_state=9)
test.to_csv('test.csv')
data = train

In [340]:
def dataCVforLinearModels(data,target,log):
    training, testing = train_test_split(data, test_size=0.2, random_state=3)
    if log == 'log':
        training = training.set_index([list(range(len(training)))])
        for i in range(len(training["SalePrice"])):
            training.loc[i,('SalePrice')] = math.log(training.loc
                                                     [i,('SalePrice')])
        testing = testing.set_index([list(range(len(testing)))])
        for i in range(len(testing["SalePrice"])):
            testing.loc[i,('SalePrice')] = math.log(testing.loc
                                                    [i,('SalePrice')])
        test_x=testing.drop(['SalePrice'],axis = 1)
        test_y=testing['SalePrice']
    X = training.drop([target], axis=1)
    Y = training[target]
    test_x=testing.drop([target], axis=1)
    test_y=testing[target]
    return training,testing,X,Y,test_x,test_y

def linearRegression(data,target):
    lm = smf.ols(target+' ~ '+' + '.join(list(data.drop(
        [target], axis=1))), data=data).fit()
    return lm

def logisticRegression(data,target):
    lr = LogisticRegression(penalty='l1', solver='liblinear')
    lr.fit(data.drop([target],axis=1),data[target])
    return lr

def CVforRegModels(data,target,lm,log):
    insample_r2=[]
    outsample_r2=[]
    outsample_rsme=[]
    for i in range (5):
        training,testing,X,Y,test_x,test_y = dataCVforLinearModels(
            data,target,log)
        if lm == "linear":
            lm = linearRegression(training,target)
        elif lm == "logistic": 
            lm = logisticRegression(training,target)
        rsq = 1. - np.sum((lm.predict(X)-Y)**2)/np.sum((
        Y.mean()-Y)**2)
        insample_r2.append(rsq)
        rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
        test_y.mean()-test_y)**2)
        outsample_r2.append(rsq)
        rsme = sqrt(mean_squared_error(test_y, lm.predict(test_x)))
        outsample_rsme.append(rsme)
    print "Estimated out of sample Root mean squared error for this model is ", sum(
        outsample_rsme)/5
    print 'Estimated out of sample R-square for this model is',sum(
        outsample_r2)/5
    print 'Estimated in sample R-square for this model is',sum(
        insample_r2)/5
    
def CVforClassificationModels(data,target,lm,log):
    insample_r2=[]
    outsample_r2=[]
    outsample_rsme=[]
    for i in range (5):
        training,testing,X,Y,test_x,test_y = dataCVforLinearModels(
            data,target,log)
        if lm == "linear":
            lm = linearRegression(training,target)
        elif lm == "logistic": 
            lm = logisticRegression(training,target)
        ab = sum((lm.predict(X)>0.5)*1*Y )/sum(Y)
        insample_r2.append(ab)
        ac = sum((lm.predict(test_x)>0.5)*1*test_y )/sum(test_y)
        outsample_r2.append(ac)
    print 'Estimated out of sample True Positive Rate for this model is',sum(
        outsample_r2)/5
    print 'Estimated in sample True Positive Rate for this model is',sum(
        insample_r2)/5
    
def FinalClassificationTest(data,target,lm,log):
    insample_r2=[]
    outsample_r2=[]
    outsample_rsme=[]
    for i in range (5):
        training,testing,X,Y,test_x,test_y = dataCVforLinearModels(
            data,target,log)
        if lm == "linear":
            lm = linearRegression(training,target)
        elif lm == "logistic": 
            lm = logisticRegression(training,target)
        ac = sum((lm.predict(test_x)>0.5)*1*test_y )/sum(test_y)
        outsample_r2.append(ac)
    print 'Final test True Positive Rate for this model is',sum(
        outsample_r2)/5

def FinalRegressionTest(data,target,lm,log):
    insample_r2=[]
    outsample_r2=[]
    outsample_rsme=[]
    for i in range (5):
        training,testing,X,Y,test_x,test_y = dataCVforLinearModels(
            data,target,log)
        if lm == "linear":
            lm = linearRegression(training,target)
        elif lm == "logistic": 
            lm = logisticRegression(training,target)
        rsq = 1. - np.sum((lm.predict(test_x)-test_y)**2)/np.sum((
        test_y.mean()-test_y)**2)
        outsample_r2.append(rsq)
        rsme = sqrt(mean_squared_error(test_y, lm.predict(test_x)))
        outsample_rsme.append(rsme)
    print "Estimated out of sample Root mean squared error for this model is ", sum(
        outsample_rsme)/5
    print 'Estimated out of sample R-square for this model is',sum(
        outsample_r2)/5

## pt a Classfication

In [319]:
CVforClassificationModels(data.drop([
    'SalePrice'], axis=1),'SaleOver2k','linear','none')
GetBiasVarianceforClassificationModels(data.drop([
    'SalePrice'], axis=1),'SaleOver2k','linear','none')

Estimated out of sample True Positive Rate for this model is 0.8287671232876711
Estimated in sample True Positive Rate for this model is 0.8711433756805806
Bias of the model is 0.009076348104644971
Variance of the model is 0.07889125799573561


In [312]:
lm = logisticRegression(data.drop([
    'SalePrice'], axis=1),'SaleOver2k')
lm_mean = (lm.predict(data.drop([
    'SalePrice','SaleOver2k'], axis=1))>0.5).mean()
dt_mean = data['SaleOver2k'].mean()
rt = abs((lm_mean-dt_mean)/dt_mean)*100
print "model prediction mean is " + str(lm_mean)
print "difference ratio is " + str(rt) +"%"

model prediction mean is 0.2937660119555935
difference ratio is 1.2912482065997204%


## pt b 

##### 1-0 Loss function is not a useful point of reference in this case for example: it might not punish the price
##### large enough and a sale price way below 200,000 and one that is close to 200,000 are totally different, but 1-0
##### loss function rates them the same way

## pt c classification

In [267]:
X = data.drop(['SalePrice','SaleOver2k'], axis=1)
Y = data['SaleOver2k']
alpha=np.arange(20,40,1)
singlefeatures = list(X)
lasso = linear_model.LassoCV(alphas=alpha,cv =5).fit(X,Y)
Importantfeatures = np.array(
    singlefeatures)[np.abs(lasso.coef_)>1e-8]
print "these vars are selected" + str(
    list(Importantfeatures))

these vars are selected[u'LotArea', u'TotalBsmtSF', u'GrLivArea', u'GarageArea']


In [268]:
Importantfeatures = list(set(Importantfeatures))
Importantfeatures.append('SaleOver2k')

In [269]:
CVforClassificationModels(
    data[Importantfeatures],'SaleOver2k','logistic',"none")

Estimated out of sample True Positive Rate for this model is 0.7411764705882353
Estimated in sample True Positive Rate for this model is 0.7146892655367232


#### they do not having a great in-sample score and 
#### out-sample is almost random. It does not perform as
#### good as baseline model (almost random)

# Q2 Buillding Models
## Best Regression Model
### Please see reasoning on the report

In [328]:
#baseline
CVforClassificationModels(
    data.drop(['SalePrice'], axis=1),'SaleOver2k','linear','none')

Estimated out of sample True Positive Rate for this model is 0.8287671232876711
Estimated in sample True Positive Rate for this model is 0.8711433756805806


In [321]:
#knn
from sklearn.neighbors import KNeighborsClassifier 
train, test = train_test_split(
    data, test_size=0.2,random_state=3)
neigh = KNeighborsClassifier(n_neighbors=7) 
neigh.fit(train.drop(['SalePrice','SaleOver2k'],axis=1), train['SaleOver2k']) 
a = sum(neigh.predict(test.drop([
    'SalePrice','SaleOver2k'],axis=1))*test['SaleOver2k'])/sum(train['SaleOver2k'])
print 'Estimated out of sample accuracy for this model is ' + str(a)

Estimated out of sample accuracy for this model is 0.17967332123411978


In [273]:
#selected variable model
data2 = pd.read_excel("NewData.xlsx")
SaleOver2k = (np.array([
    data2['SalePrice']>200000])*1).reshape(len(data2),1)
data2= data2.assign(SaleOver2k = SaleOver2k)
singlefeatures = data2.columns[:-2]
features = list(singlefeatures)
for i in range(len(features)):
    features[i] = "".join(features[i].split())
    features[i] = features[i].replace('.','DOT')
singlefeatures = features
features.append('SalePrice')
features.append('SaleOver2k')
data2.columns = features

In [274]:
#selected variable model
CVforClassificationModels(
    data2.drop(['SalePrice'],axis = 1),'SaleOver2k','linear','none')

Estimated out of sample True Positive Rate for this model is 0.8469945355191257
Estimated in sample True Positive Rate for this model is 0.8382789317507419


In [275]:
#all variables with logistic model
CVforClassificationModels(
    data.drop(['SalePrice'],axis = 1),'SaleOver2k','logistic','none')
(data.drop(['SalePrice'], axis=1),'SaleOver2k','linear','none')

Estimated out of sample True Positive Rate for this model is 0.9176470588235294
Estimated in sample True Positive Rate for this model is 0.940677966101695


In [276]:
data = data2

In [277]:
data['ExterQual'] = data['ExterQual'].replace('Ex', 5)
data['ExterQual'] = data['ExterQual'].replace('Gd', 4)
data['ExterQual'] = data['ExterQual'].replace('TA', 3)
data['ExterQual'] = data['ExterQual'].replace('Fa', 2)
data['ExterQual'] = data['ExterQual'].replace('Po', 1)
data['ExterCond'] = data['ExterCond'].replace('Ex', 5)
data['ExterCond'] = data['ExterCond'].replace('Gd', 4)
data['ExterCond'] = data['ExterCond'].replace('TA', 3)
data['ExterCond'] = data['ExterCond'].replace('Fa', 2)
data['ExterCond'] = data['ExterCond'].replace('Po', 1)
data['BsmtQual'] = data['BsmtQual'].replace('Ex', 5)
data['BsmtQual'] = data['BsmtQual'].replace('Gd', 4)
data['BsmtQual'] = data['BsmtQual'].replace('TA', 3)
data['BsmtQual'] = data['BsmtQual'].replace('Fa', 2)
data['BsmtQual'] = data['BsmtQual'].replace('Po', 1)
data['BsmtQual'] = data['BsmtQual'].replace('No Basement', 0)
data['BsmtCond'] = data['BsmtCond'].replace('Ex', 5)
data['BsmtCond'] = data['BsmtCond'].replace('Gd', 4)
data['BsmtCond'] = data['BsmtCond'].replace('TA', 3)
data['BsmtCond'] = data['BsmtCond'].replace('Fa', 2)
data['BsmtCond'] = data['BsmtCond'].replace('Po', 1)
data['BsmtCond'] = data['BsmtCond'].replace('No Basement', 0)

In [278]:
#Selected Variable with Logistic regression
CVforClassificationModels(
    data.drop(['SalePrice'],axis=1),'SaleOver2k','logistic',"none")

Estimated out of sample True Positive Rate for this model is 0.8579234972677595
Estimated in sample True Positive Rate for this model is 0.870919881305638


In [283]:
#Lasso Features with Logistic regression
X = data.drop(['SaleOver2k','SalePrice'], axis=1)
Y = data['SaleOver2k']
alpha=np.arange(0,150,0.1)
singlefeatures = list(X)
lasso = linear_model.LassoCV(alphas=alpha,cv =5).fit(X,Y)
lassofeatures = np.array(
    singlefeatures)[np.abs(lasso.coef_)>1e-8]
lassofeatures = list(lassofeatures)
lassofeatures.append('SaleOver2k')

In [291]:
#Lasso Features with Logistic regression
CVforClassificationModels(
    data[lassofeatures],'SaleOver2k','logistic','none')

Estimated out of sample True Positive Rate for this model is 0.8424657534246576
Estimated in sample True Positive Rate for this model is 0.925589836660617


# Q3 Prediction Classfication

In [332]:
test = pd.read_csv("test.csv")

In [290]:
#all variables with logistic model(best)
FinalClassificationTest(
    data.drop(['SalePrice'],axis = 1),'SaleOver2k','logistic','none')

Final test True Positive Rate for this model is 0.8424657534246576


In [294]:
#baseline_linear
FinalClassificationTest(
    data.drop(['SalePrice'],axis = 1),'SaleOver2k','linear','none')

Final test True Positive Rate for this model is 0.8287671232876711
